In [1]:
from inspect import getsourcefile
import os.path
import sys
import time
current_path = os.path.abspath(getsourcefile(lambda:0))
parent_dir = os.path.split(os.path.dirname(current_path))[0]
sys.path.insert(0, parent_dir)
import common.load_notebooks
import common.utils

importing Jupyter notebook from common/utils.ipynb


<IPython.core.display.Javascript object>

In [2]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
print('done')

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
done


In [3]:

n_input=784
n_classes=10
training_iters = 200000
batchsize=128
learning_rate = 0.001
dropout = 0.75 
display_step = 10
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)
def wireup(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    c1 = tf.nn.conv2d(x, weights['c1'], strides=[1,1,1,1], padding='SAME')
    c1 = tf.nn.bias_add(c1, biases['bc1'])
    c1 = tf.nn.max_pool(c1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    c2 = tf.nn.conv2d(c1, weights['c2'], strides=[1,1,1,1], padding='SAME')
    c2 = tf.nn.bias_add(c2, biases['bc2'])
    c2 = tf.nn.max_pool(c2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    fc1 = tf.reshape(c2, [-1, weights['f1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['f1']), biases['bf1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    out = tf.add(tf.matmul(fc1, weights['out']), biases['bout'])
    return out


    

In [4]:

# Initializing the variables
weights = {
    'c1' :  tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'c2' : tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'f1' : tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out' :  tf.Variable(tf.random_normal([1024, n_classes])),
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([32])),
    'bc2' : tf.Variable(tf.random_normal([64])),
    'bf1' : tf.Variable(tf.random_normal([1024])),
    'bout' : tf.Variable(tf.random_normal([n_classes])),
}
pred = wireup(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.initialize_all_variables()

In [5]:
from IPython.display import display
p=common.utils.ProgressImageWidget()
display(p)
with tf.Session() as sess:
    sess.run(init)
    step = 1
    plotter = common.utils.Plotter(
        xlabel='no. of iterations',
        ylabel='accuracy',
        title='iterations vs. accuracy')
    plotter.add_channel(num_samples=int(np.floor(training_iters/(batchsize  * display_step))),
                        channel_name='train',
                        legend='train')
    plotter.add_channel(num_samples=int(np.floor(training_iters/(batchsize  * display_step* 10))),
                        channel_name='test',
                        legend='test')
    
    while step*batchsize < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batchsize)
        sess.run(optimizer, feed_dict={x:batch_x, y:batch_y, keep_prob:dropout})
        if step % display_step == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            plotter.add_sample( step,  acc, channel_name='train')
            if step % (display_step * 10) == 0:
                plotter.add_sample( step,  acc * 0.8, channel_name='test')
            p.value = plotter.plot()
        step += 1
    print('done')
        

done
